# Algorytm k najbliższych sąsiadów

W ćwiczeniu wykorzystywany jest zbiór danych `donors.csv`, opracowany przez organizację weteranów, która zabiga o datki, kierując kampanie listowe do aktualnych i potencjalnych darczyńców z bazy danych. Zbiór danych zawiera informacje o cechach charakterystycznych klientów oraz o tym, czy zareagowali na kampanię testową. Zadaniem jest pomoc organizacji w przewidzeniu, którzy darczyńcy odpowiedzą na kampanię listową w oparciu o dane demograficzne, wcześniejszą historię datków oraz reakcję na poprzednie kampanie listowe.

1. Zaimportuj dane i wyświetl ich podgląd (funkcje `read_csv` oraz `glimpse`. Pierwsze 12 zmiennych jest typu `numeric`, pozostałe 10 jest typu `factor`). Cecha klasy nosi nazwę `respondentMailing`.

In [1]:
library(tidyverse)
options(jupyter.rich_display = FALSE,
        repr.plot.width = 15,
        repr.plot.height = 8)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
df <- read_csv("data/donors.csv")

Rows: 95412 Columns: 22
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (4): state, urbanicity, socioEconomicStatus, gender
dbl (12): age, numberChildren, incomeRating, wealthRating, mailOrderPurchase...
lgl  (6): inHouseDonor, plannedGivingDonor, sweepstakesDonor, P3Donor, isHom...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [3]:
head(df)

  age numberChildren incomeRating wealthRating mailOrderPurchases
1 60  NA             NA           NA            0                
2 46   1              6            9           16                
3 NA  NA              3            1            2                
4 70  NA              1            4            2                
5 78   1              3            2           60                
6 NA  NA             NA           NA            0                
  totalGivingAmount numberGifts smallestGiftAmount largestGiftAmount
1 240               31           5                 12               
2  47                3          10                 25               
3 202               27           2                 16               
4 109               16           2                 11               
5 254               37           3                 15               
6  51                4          10                 16               
  averageGiftAmount ⋯ inHouseDonor plannedGivingDonor s

In [4]:
dim(df)

[1] 95412    22

In [5]:
df <- df %>%
  mutate(
         age = as.integer(age),
         numberChildren = as.integer(numberChildren),
         incomeRating = as.integer(incomeRating),
         wealthRating = as.integer(wealthRating),
         mailOrderPurchases = as.integer(mailOrderPurchases),
         totalGivingAmount = as.integer(totalGivingAmount),
         numberGifts = as.integer(numberGifts),
         smallestGiftAmount = as.integer(smallestGiftAmount),
         largestGiftAmount = as.integer(largestGiftAmount),
         averageGiftAmount = as.double(averageGiftAmount),
         yearsSinceFirstDonation = as.integer(yearsSinceFirstDonation),
         monthsSinceLastDonation = as.integer(monthsSinceLastDonation),
         inHouseDonor = as.factor(inHouseDonor),
         plannedGivingDonor = as.factor(plannedGivingDonor),
         sweepstakesDonor = as.factor(sweepstakesDonor),
         P3Donor = as.factor(P3Donor),
         state = as.factor(state),
         urbanicity = as.factor(urbanicity),
         socioEconomicStatus = as.factor(socioEconomicStatus),
         isHomeowner = as.factor(isHomeowner),
         gender = as.factor(gender), 
         respondedMailing = as.factor(respondedMailing))

In [6]:
glimpse(df)

Rows: 95,412
Columns: 22
$ age                     <int> 60, 46, NA, 70, 78, NA, 38, NA, NA, 65, NA, 75…
$ numberChildren          <int> NA, 1, NA, NA, 1, NA, 1, NA, NA, NA, NA, NA, 2…
$ incomeRating            <int> NA, 6, 3, 1, 3, NA, 4, 2, 3, NA, 2, 1, 4, NA, …
$ wealthRating            <int> NA, 9, 1, 4, 2, NA, 6, 9, 2, NA, 0, 5, 2, NA, …
$ mailOrderPurchases      <int> 0, 16, 2, 2, 60, 0, 0, 1, 0, 0, 0, 3, 16, 0, 1…
$ totalGivingAmount       <int> 240, 47, 202, 109, 254, 51, 107, 31, 199, 28, …
$ numberGifts             <int> 31, 3, 27, 16, 37, 4, 14, 5, 11, 3, 1, 2, 9, 1…
$ smallestGiftAmount      <int> 5, 10, 2, 2, 3, 10, 3, 5, 10, 3, 20, 10, 4, 5,…
$ largestGiftAmount       <int> 12, 25, 16, 11, 15, 16, 12, 11, 22, 15, 20, 15…
$ averageGiftAmount       <dbl> 7.741935, 15.666667, 7.481481, 6.812500, 6.864…
$ yearsSinceFirstDonation <int> 8, 3, 7, 10, 11, 3, 10, 3, 9, 3, 1, 1, 8, 5, 4…
$ monthsSinceLastDonation <int> 14, 14, 14, 14, 13, 20, 22, 18, 19, 22, 12, 14…
$ inHouseDonor 

2. Ogranicz predyktory wyłącznie do cech liczbowych ze zbioru danych (funkcja `select`). Wyświetl podsumowanie statystyczne dla nowego zbioru danych i sprawdź dla których zmiennych występują braki danych.

In [7]:
df_numeric <- df %>%
    select(where(is.numeric))

df_numeric["respondedMailing"] <- df["respondedMailing"]

In [8]:
numberofna <- df_numeric %>%
  filter(if_any(everything(), is.na)) %>%
  nrow()

In [9]:
round(numberofna / nrow(df_numeric) * 100, 2)

[1] 91.86

*92% wierszy w kolumnach numerycznych (1:12) ma w sobie przynajmniej jedno `NA`*

3. Uzupełnij brakujące dane: dla zmiennej `age` użyj imputacji średniej, dla zmiennej `numberChildren` - imputacji mediany. W przypadku cech `incomeRating` oraz `wealthRating` problem brakujących danych rozwiąż wykluczając te wystąpienia ze zbioru danych. Ponadto dla zmiennej `wealthRating` (miara ogólnego majątku darczyńcy w skali od 1 do 9) należy również wykluczyć wystąpienia o wartości 0.

In [10]:
df_numeric["age"][is.na(df_numeric["age"])] <- as.integer(mean(df_numeric$age, na.rm = TRUE))
df_numeric["numberChildren"][is.na(df_numeric["numberChildren"])] <- median(df_numeric$numberChildren, na.rm = TRUE)

df_numeric <- df_numeric %>%
    filter(!is.na(incomeRating) & 
           !is.na(wealthRating) & 
           wealthRating > 0
    )

In [11]:
df_numeric %>%
  filter(if_any(everything(), is.na)) %>%
  nrow()

[1] 0

In [12]:
dim(df_numeric)

[1] 48267    13

*nie ma braków juz, zostało 48_267 wierszy*

4. Utwórz funkcję normalizacji min-max i zapisz ją jako `normalize`. Następnie wykorzystaj ją do znormalizowania wartości cech (ustandaryzowania skali do zakresu od 0 do 1).

In [13]:
min_max <- function(x) {
    return ( (x - min(x)) / (max(x) - min(x)) )
}

In [14]:
df_numeric_norm <- data.frame(sapply(df_numeric, function(x) if (is.numeric(x)) min_max(x) else x))
df_numeric_norm["respondedMailing"] <- as.factor(df_numeric_norm["respondedMailing"] == 2)
# podczas stosowania sapply do minmaxa moj factor z jakiegos powodu przerzucil sie numeric z wartosciami 1 i 2, wiec zmieniam na factor z powrotem

5. Podziel dane na zbiory treningowy i testowy w stosunku 75:25, przekształcając uprzednio dane w ramkę danych. Porównaj rozkłady klas dla oryginalnego zbioru danych oraz zbiorów treningowego i testowego.

In [15]:
set.seed(2024)
split <- rsample::initial_split(df_numeric_norm, prop = 0.75)

X_train <- rsample::training(split)
X_test <- rsample::testing(split)

In [16]:
print_class_dist <- function(data, name) {
    cat(glue::glue("{name} dataset class distribution"))
    data %>%
    group_by(respondedMailing) %>%
    count(name = "count") %>%
    mutate(perc_dist = count/nrow(data))
}

In [17]:
print_class_dist(df, "Whole")

Whole dataset class distribution

  respondedMailing count perc_dist 
1 FALSE            90569 0.94924119
2 TRUE              4843 0.05075881

In [18]:
print_class_dist(X_train, "Training")

Training dataset class distribution

  respondedMailing count perc_dist 
1 FALSE            34325 0.94820442
2 TRUE              1875 0.05179558

In [19]:
print_class_dist(X_test, "Test")

Test dataset class distribution

  respondedMailing count perc_dist 
1 FALSE            11445 0.94845446
2 TRUE               622 0.05154554

6. Za pomocą funkcji `smote` z pakietu `performanceEstimation` zrównoważ dane treningowe.

In [20]:
library(performanceEstimation)
smote_data <- smote(respondedMailing ~ ., data = X_train, perc.over = 9, perc.under = 1.1, k = 5)
print_class_dist(smote_data, "Smoted")

nrow(smote_data)
nrow(X_train)

Smoted dataset class distribution

  respondedMailing count perc_dist
1 FALSE            18562 0.4974807
2 TRUE             18750 0.5025193

[1] 37312

[1] 36200

*chcialem zrobic tak, zeby zostalo 34k falsow i dodac 32k trueow, ale przekroczylo to 16gb pamieci, wiec postanowilem wyrownac zeby bylo +- tyle samo obserwacji*

In [21]:
X_train <- smote_data

7. Przenieś etykiety klas do osobnych zbiorów danych. Z pomocą polecenia `pull()` z pakietu `tidyverse` utwórz nowe wektory z etykietami cechy klasy (`respondedMailing`) i przekształć je do typu `factor`.

8. Przekształć zbiory danych treningowych i testowych do postaci ramek danych bez etykiet klas (`respondedMailing`).

*pewnie madrze byloby uzyc pull, ale z racji ze mam klasy w ostatniej kolumnie i juz sa typu factor to po prostu je po chlopsku wyciagne i usune ostatnia kolumne*

In [22]:
y_train <- X_train$respondedMailing
y_test <- X_test$respondedMailing

X_train <- X_train[-13]
X_test <- X_test[-13]

9. Budowanie modelu: Wykorzystaj funkcję `knn()` z pakietu `class` do oznaczenia przykładów testowych na podstawie danych treningowych. `k` ustaw wartość `5`.

In [23]:
library(class)

In [24]:
y_pred <- knn(train = X_train, test = X_test, cl = y_train, k = 5)

10. Wyświetl pierwsze 6 predykcji.

In [25]:
head(y_pred, 6)

[1] FALSE TRUE  FALSE FALSE TRUE  FALSE
Levels: FALSE TRUE

*fajnie ze od razu mamy factor po predykcji nie trzeba rzutowac*

11. Utwórz macierz pomyłek i na jej podstawie oblicz dokładność predykcji.

In [26]:
caret::confusionMatrix(y_pred, y_test)

Confusion Matrix and Statistics

          Reference
Prediction FALSE TRUE
     FALSE  8031  427
     TRUE   3414  195
                                        
               Accuracy : 0.6817        
                 95% CI : (0.6733, 0.69)
    No Information Rate : 0.9485        
    P-Value [Acc > NIR] : 1             
                                        
                  Kappa : 0.0047        
                                        
 Mcnemar's Test P-Value : <2e-16        
                                        
            Sensitivity : 0.70170       
            Specificity : 0.31350       
         Pos Pred Value : 0.94952       
         Neg Pred Value : 0.05403       
             Prevalence : 0.94845       
         Detection Rate : 0.66553       
   Detection Prevalence : 0.70092       
      Balanced Accuracy : 0.50760       
                                        
       'Positive' Class : FALSE         
                                        

12. Zmodyfikuj kod tak, aby użyć zmiennych kategorialnych w celu poprawy dokładności modelu.

In [27]:
numberofna <- df %>%
  filter(if_any(everything(), is.na)) %>%
  nrow()
  
round(numberofna / nrow(df) * 100, 2)

[1] 92.71

*93% wierszy we wszystkich kolumnach (1:22) ma w sobie przynajmniej jedno `NA`*

In [28]:
df["age"][is.na(df["age"])] <- as.integer(mean(df$age, na.rm = TRUE))
df["numberChildren"][is.na(df["numberChildren"])] <- median(df$numberChildren, na.rm = TRUE)

df <- df %>%
    filter(!is.na(incomeRating) & 
           !is.na(wealthRating) & 
           wealthRating > 0
    )

In [29]:
df %>%
  filter(if_any(everything(), is.na)) %>%
  nrow()

[1] 13702

*po tych operacjach na kolumnach numerycznych (1:12) bylo 0 NA, co oznacza, ze mamy 13k wierszy, ktore w kolumnach kategorialnych (13:22) maja przynajmniej jedno NA, troche przypal*

In [30]:
summary(df[13:22])

 inHouseDonor  plannedGivingDonor sweepstakesDonor  P3Donor     
 FALSE:44542   FALSE:48225        FALSE:47467      FALSE:47249  
 TRUE : 3725   TRUE :   42        TRUE :  800      TRUE : 1018  
                                                                
                                                                
                                                                
                                                                
                                                                
     state        urbanicity    socioEconomicStatus isHomeowner     gender     
 CA     : 7999   city  :10668   average:24616       TRUE:35856   female:25412  
 FL     : 4585   rural : 7589   highest:16598       NA's:12411   joint :  215  
 TX     : 3894   suburb:12613   lowest : 5925                    male  :21644  
 IL     : 3529   town  :10558   NA's   : 1128                    NA's  :  996  
 MI     : 3005   urban : 5711                                                  


*wyglada na to, ze cos zlego stalo sie z kolumna isHomeowner, gdy mamy tam tylko 35k TRUE, a reszta 12k to NA. podejrzewam, ze moze byc tak, ze te NA to tak naprawde FALSE, ale pewnosci nie mam i moze to zaburzyc knn wiec na wszelki wypadek pozbede sie tej kolumny z dalszych obliczen*

In [31]:
df <- df[-20]
numberofna <- df %>%
  filter(if_any(everything(), is.na)) %>%
  nrow()
  
round(numberofna / nrow(df) * 100, 2)

[1] 4.35

*teraz mamy 4.35% brakow, mozna usuwac*

In [32]:
df <-df %>%
   filter(!if_any(everything(), is.na))

In [33]:
df_norm <- cbind(data.frame(sapply(df[1:12], function(x) min_max(x))),  # numerical cols normalized 
                 model.matrix(~ . - 1, data = df[13:20]),  # cathegorical cols one-hot-encoded
                 df[21])  # class column as factor without changes

In [34]:
any(is.na(df_norm))

[1] FALSE

In [35]:
set.seed(2024)
split <- rsample::initial_split(df_norm, prop = 0.75)

X_train <- rsample::training(split)
X_test <- rsample::testing(split)

In [36]:
print_class_dist(df, "Whole")

Whole dataset class distribution

  respondedMailing count perc_dist
1 FALSE            43779 0.9482542
2 TRUE              2389 0.0517458

In [37]:
print_class_dist(X_train, "Train")

Train dataset class distribution

  respondedMailing count perc_dist
1 FALSE            32853 0.9487957
2 TRUE              1773 0.0512043

In [38]:
print_class_dist(X_test, "Test")

Test dataset class distribution

  respondedMailing count perc_dist
1 FALSE            10926 0.9466297
2 TRUE               616 0.0533703

In [39]:
smote_data <- smote(respondedMailing ~ ., data = X_train, perc.over = 9, perc.under = 1.1, k = 5)
print_class_dist(smote_data, "Smoted")

nrow(smote_data)
nrow(X_train)

Smoted dataset class distribution

  respondedMailing count perc_dist
1 FALSE            17552 0.4974775
2 TRUE             17730 0.5025225

[1] 35282

[1] 34626

In [40]:
X_train <- smote_data

In [41]:
y_train <- X_train$respondedMailing
y_test <- X_test$respondedMailing

X_train <- X_train[-82]
X_test <- X_test[-82]

In [42]:
y_pred <- knn(train = X_train, test = X_test, cl = y_train, k = 5)

In [43]:
caret::confusionMatrix(y_pred, y_test)

Confusion Matrix and Statistics

          Reference
Prediction FALSE TRUE
     FALSE  9539  518
     TRUE   1387   98
                                          
               Accuracy : 0.835           
                 95% CI : (0.8281, 0.8417)
    No Information Rate : 0.9466          
    P-Value [Acc > NIR] : 1               
                                          
                  Kappa : 0.0193          
                                          
 Mcnemar's Test P-Value : <2e-16          
                                          
            Sensitivity : 0.87306         
            Specificity : 0.15909         
         Pos Pred Value : 0.94849         
         Neg Pred Value : 0.06599         
             Prevalence : 0.94663         
         Detection Rate : 0.82646         
   Detection Prevalence : 0.87134         
      Balanced Accuracy : 0.51607         
                                          
       'Positive' Class : FALSE           
                     

*jest lepiej! fajnie :)*

In [44]:
# EOF